In [1]:
import nltk
import pandas as pd
import sys
import os
import io
import operator

from data_loader import DataLoader

from nltk.tokenize import word_tokenize ,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from nltk import pos_tag

import math

import numpy

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import networkx as nx

import re

import networkx as nx

from collections import defaultdict
import string

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import copy
import itertools

import pke

from  itertools import chain

import random
random.seed(0)

In [2]:
loader = DataLoader(DataLoader.data_path1)
table = loader.load_table()
table.head()

C:\Users\lijia\Desktop\MSAI\Sem A\AI6122\Assignment1\Hearts-of-Iron\data_loader.py:14: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);


200
200


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
13005,A1Q5BYJHCI46Q4,B0037BS2IQ,1313,"[0, 0]","Great read, as is all of Scarlet's books! Can'...",5,Life Class...,1361836800,"02 26, 2013"
13006,A301X46BHB2WD8,B0037BS2IQ,"Bogey ""Gene""","[3, 4]","I thoroughly enjoyed ""Life Class"". This is a ...",5,Art at it's best,1272672000,"05 1, 2010"
13007,A3O9A7B0KJO88B,B0037BS2IQ,Charly T. Anchor,"[3, 3]",Warning: This review might contain what some p...,3,Angsty story with good buildup but a disappoin...,1304553600,"05 5, 2011"
13008,A1E481YU3ZPK6S,B0037BS2IQ,Elisa,"[3, 3]","The cover is really good, and indeed, if the o...",5,Life Class by Scarlet Blackwell,1278374400,"07 6, 2010"
13009,A3P4V9UX4YMZ1W,B0037BS2IQ,"L. Curtis ""Lynette's Two Cents Reviews""","[2, 2]",I closed LIFE CLASS with a sense of satisfacti...,4,Touching and Emotional,1273276800,"05 8, 2010"


In [3]:
#Data Preprocess
#Get stopwords
nltk_stopwords = stopwords.words('english')
nltk_stopwords.append('\n')
spacy_stopwords = list(STOP_WORDS)
merged_sw = nltk_stopwords + list(set(spacy_stopwords) - set(nltk_stopwords))

print("nltk: ", len(nltk_stopwords))
print("spacy: ", len(spacy_stopwords))
print("merge: ", len(merged_sw))
print(merged_sw)

nltk:  180
spacy:  326
merge:  383
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'on

In [4]:
#Create the review text dictionary
reviewText = {}
for i in table.index:
    if table['asin'][i] in reviewText:
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
    else:
        reviewText[table['asin'][i]] = []
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)

In [5]:
def data_preprocess(original_review, stop_words):
    # to lowercase
    review = original_review.lower()
    # remove punctuation
    review = re.sub(r'[^\w\s]', ' ', review).strip()

    lemmatizer = WordNetLemmatizer()
    review_words = review.split(' ')
    review_words_tag = nltk.pos_tag(review_words)
    processed_review = ""
    # remove stop words&lemma
    for word, tag in review_words_tag:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma_word = word
        else:
            lemma_word = lemmatizer.lemmatize(word, wntag)
        if lemma_word not in stop_words:
            processed_review = processed_review + lemma_word + " "
    processed_review = " ".join(processed_review.split())
    processed_review = processed_review.strip()
    return processed_review



In [8]:
processed_reviewText = {}
for r in reviewText:
    processed_reviewText[r] = []
    for t in reviewText[r]:
        p = data_preprocess(t, merged_sw)
        processed_reviewText[r].append(p)
print(processed_reviewText)

{'B0037BS2IQ': ['great read scarlet book', 'description share enjoy', 'check win desappointed', 'thoroughly enjoy life class', 'write story 2 people attract different way', 'sorry end suddenly explain', 'short story character develop plot action flow', 'hope blackwell consider write sequel develop personality harley especially ryan', 'warn review contain people consider spoiler rating 6 10pros tentative forth nature men relationship start tender nice change seduction portray romance harley struggle mightily desire ryan love harley suffer long', 'like read character sexual restraint guy start physical relationship bedroom scene angry variety', 'blackwell capture tone episode', 'minute men snarl shove snarl kiss con guy constantly hurtful offensive thing', 'instance fact find guy simply despicable ryan constant waver feeling harley induce whiplash mean constant hat desire pity admire harley span single scene transition ryan hate harley cruel ryan love harley want live incredibly abrupt',

In [9]:
# collect frequency

def collect_frequency(processed_review):
    word_frequency = defaultdict(lambda:0)
    for v in processed_review.values():
        for i in v:
            tokens = nltk.word_tokenize(i)
            for token in tokens:
                word_frequency[token] += 1  
    return word_frequency

frequency = collect_frequency(processed_reviewText)

In [10]:
frequency = sorted(frequency.items(), key=lambda item: item[1],reverse=True)
print(frequency)

[('book', 12479), ('read', 7814), ('story', 7063), ('love', 6415), ('like', 3940), ('character', 3536), ('good', 2963), ('series', 2941), ('time', 2908), ('great', 2584), ('find', 2489), ('want', 2459), ('end', 2343), ('know', 2215), ('author', 2185), ('think', 2144), ('enjoy', 1930), ('come', 1884), ('life', 1875), ('way', 1839), ('kindle', 1839), ('write', 1749), ('thing', 1537), ('wait', 1495), ('work', 1476), ('8217', 1456), ('look', 1427), ('little', 1383), ('34', 1308), ('short', 1301), ('need', 1289), ('start', 1272), ('recommend', 1170), ('charge', 1144), ('use', 1124), ('romance', 1112), ('lot', 1105), ('feel', 1053), ('leave', 1041), ('happen', 1022), ('try', 1009), ('review', 946), ('new', 944), ('year', 924), ('fire', 874), ('man', 852), ('help', 835), ('right', 820), ('reader', 815), ('friend', 815), ('bit', 806), ('novel', 800), ('hot', 798), ('woman', 793), ('tell', 789), ('family', 783), ('plot', 778), ('charger', 771), ('star', 766), ('day', 755), ('sex', 747), ('page'

In [11]:
sorted_processed_reviewText = sorted(processed_reviewText.items(), key=lambda item: len(item[1]),reverse=True)
print(sorted_processed_reviewText[:3])

[('B006GWO5WK', ['absolute necessity', 'work great speed need recharge kindle', 'carry kindle plug need', 'charger charger matter good specially kindle use charge need', 'run power hour charge kindle wonderful product charge battery hour', 'great investment', 'charger include buy product', 'greedy amazon', 'charger', 'good charger little pricey box kindle', 'like way prong fold flat travel', 'agree charger include purchase kindle fire hd 9 99 worth extra cost', 'try charge usb cord include incredibly long time', '14 hour powerfast charger 5 hour fully charge', 'find odd charger include kindle fire happy powerfast charger', 'kindle fire need charge read appreciate device charge fast easy', 'amazon kindle powerfast accelerated charge sell 19 99 typically 9 99 buy kindle', 'charger include kindle bit silly opinion amazon try price', 'buy charger trickle charge usb drive like watch turtle race', 'excruciating thing dial internet', 'suck buy charger', 'win regret time save', 'amazon thankyo

In [12]:
# #DM B0007NFL18 3027> B00006690F 2625 > B0000AGWFX 2404
# print(len(processed_reviewText['B0007NFL18']))

In [22]:
#KS B006GWO5WK 3495> B0078S9B6G 1771 > B005C5YZ86 775
print(len(processed_reviewText['B0078S9B6G']))

1771


In [23]:
# Abstractive Summarizer by Model
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = 'google/pegasus-xsum'
torch_device = 'cuda'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

text = """"""
for i in reviewText['B0078S9B6G']:
    text+=i
print(text)

batch = tokenizer.prepare_seq2seq_batch(text, truncation=True, padding='longest',return_tensors='pt').to(torch_device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

print(tgt_text)

The Mine has its ups and downs but in the end it comes off as a truly entertaining, adventure mixed with historical fact and sprinkled with a dash of romance.College senior Joel Smith is inadvertently cast as the protagonist in this exciting, time traveling story, so one could call him the reluctant protagonist.He's just a regular guy who finds himself on an irregular journey that anyone can relate to.I don't know if author John Heldt intended Joel to come across as such, but there it is.Now before you go thinking "Eh, this is just another time traveling sci-fi book, I'll pass" let me just say, The Mine has emotional side.It's not just some college hotshot time traveling and having the time of his life.Joel knows a lot of the people he meets, he knows what is going to happen or not going to happen to them in the future and he is powerless to stop it.As the reader, I began to get attached to these people, just like Joel did even against my better judgement.Author John Heldt didn't write

C:\Users\lijia\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3415: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


['The Mine by John Heldt is one of those books that will leave you wanting more.']


In [24]:
#TextRank
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

keyphrases

[('love a great time travel romance', 0.025383758195899858),
 ('a good time travel story', 0.023806455132066567),
 ('a time travel book', 0.01910600248026587),
 ('a time travel story', 0.018676333251916817),
 ('enjoyable time travel story', 0.01860375472146297),
 ('typical time travel story', 0.01834438256293809),
 ('time travel romance books', 0.018208664821572063),
 ('a time travel romance', 0.017908072791107673),
 ('time travel book', 0.017891125109842494),
 ('time travel story', 0.017511585881493444)]

In [25]:
# 1. create a YAKE extractor.
extractor = pke.unsupervised.YAKE()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None,
                        stoplist=merged_sw)


# 3. select {1-3}-grams not containing punctuation marks and not
#    beginning/ending with a stopword as candidates.
extractor.candidate_selection(n=3)

# 4. weight the candidates using YAKE weighting scheme, a window (in
#    words) for computing left/right contexts can be specified.
window = 2
use_stems = False # use stems instead of words for weighting
extractor.candidate_weighting(window=window,
                              use_stems=use_stems)

# 5. get the 10-highest scored candidates as keyphrases.
#    redundant keyphrases are removed from the output using levenshtein
#    distance and a threshold.
threshold = 0.8
keyphrases = extractor.get_n_best(n=10, threshold=threshold)

keyphrases

[('joel', 0.00010119400670184326),
 ('joel smith', 0.00015472215573011043),
 ('time', 0.0005689990759070123),
 ('book', 0.000625311392197086),
 ('john heldt', 0.0006364925955359935),
 ('time travel', 0.0007095521136759349),
 ('heldt', 0.0011611792346470447),
 ('story', 0.0012932964064672559),
 ('grace', 0.0016024165345141874),
 ('joel meets grace', 0.0020159143954414033)]

In [26]:
# 1. create a TfIdf extractor.
extractor = pke.unsupervised.TfIdf()

# 2. load the content of the document.
stoplist = list(string.punctuation)
stoplist += pke.lang.stopwords.get('en')
extractor.load_document(input=text,
                        language='en',
                        stoplist=merged_sw,
                        normalization=None)

# 3. select {1-3}-grams not containing punctuation marks as candidates.
extractor.candidate_selection(n=3)

# 4. weight the candidates using a `tf` x `idf`
df = pke.load_document_frequency_file(input_file='df.tsv.gz')
extractor.candidate_weighting(df=df)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

keyphrases

[('joel', 613.3804877790875),
 ('book', 529.3774752408661),
 ('story', 358.20152516298134),
 ('love', 244.08422511105806),
 ('travel', 209.21505009519262),
 ('time travel', 187.02557508509645),
 ('like', 152.156400069231),
 ('characters', 152.156400069231),
 ('heldt', 148.9864750677887),
 ('life', 137.8917375627406)]

In [27]:
# TopicRank
# initialize keyphrase extraction model, here TopicRank
extractor = pke.unsupervised.TopicRank()

# load text
extractor.load_document(input=text, language='en')

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
extractor.candidate_selection()

# candidate weighting, in the case of TopicRank: using a random walk algorithm
extractor.candidate_weighting()

# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
keyphrases = extractor.get_n_best(n=10)

keyphrases

[('college senior joel smith', 0.041847473147580264),
 ('book', 0.034068603639587206),
 ('time', 0.019748041996034367),
 ('story', 0.019282893994443363),
 ('time travel novels', 0.013611374119451368),
 ('character', 0.013491458615274347),
 ('author', 0.012249283697213911),
 ('love story', 0.012231939869198979),
 ('friends', 0.011441811141267667),
 ('way', 0.009715676299942367)]

In [35]:
# RAKE Algorithm
def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False

def load_stop_words(stop_word_file, regex):
    with io.open(stop_word_file, encoding='utf8') as stop_word_file:
        stop_words = re.split(regex, stop_word_file.read())
    return [word for word in stop_words if word not in ('', ' ')]  # filters empty string matches


def separate_words(text):
    """
    Utility function to return a list of all words that are have a length greater than a specified number of characters.
    @param text The text that must be split in to words.
    @param min_word_return_size The minimum no of characters a word must have to be included.
    """
    splitter = re.compile(r'(?u)\W+')
    words = []
    for single_word in splitter.split(text):
        current_word = single_word.strip().lower()
        # leave numbers in phrase, but don't count as words, since they tend to invalidate scores of their phrases
        if current_word != '' and not is_number(current_word):
            words.append(current_word)
    return words


def split_sentences(text):
    """
    Utility function to return a list of sentences.
    @param text The text that must be split in to sentences.
    """
    sentence_delimiters = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s')
    sentences = sentence_delimiters.split(text)
    return sentences


def build_stop_word_regex(stop_word_list):
    stop_word_regex_list = []
    for word in stop_word_list:
        word_regex = r'\b' + word + r'(?![\w-])'
        stop_word_regex_list.append(word_regex)
    return re.compile('(?u)' + '|'.join(stop_word_regex_list), re.IGNORECASE)


def generate_candidate_keywords(sentence_list, stop_word_pattern, minCharacters, maxWords):
    phrase_list = []
    for s in sentence_list:
        tmp = re.sub(stop_word_pattern, '|', s.strip())
        phrases = tmp.split("|")
        for phrase in phrases:
            phrase = phrase.strip().lower()
            if phrase != '' and len(phrase) >= minCharacters and len(phrase.split()) <= maxWords:
                phrase_list.append(phrase)
    return phrase_list


def calculate_word_scores(phraseList):
    word_frequency = {}
    word_degree = {}
    for phrase in phraseList:
        word_list = separate_words(phrase)
        word_list_length = len(word_list)
        word_list_degree = word_list_length - 1
        for word in word_list:
            word_frequency.setdefault(word, 0)
            word_frequency[word] += 1
            word_degree.setdefault(word, 0)
            word_degree[word] += word_list_degree
    for item in word_frequency:
        word_degree[item] = word_degree[item] + word_frequency[item]

    # Calculate Word scores = deg(w)/frew(w)
    word_score = {}
    for item in word_frequency:
        word_score.setdefault(item, 0)
        word_score[item] = word_degree[item] / (word_frequency[item] * 1.0)
    return word_score


def generate_candidate_keyword_scores(phrase_list, word_score, minFrequency):
    keyword_candidates = {}
    # iterate over the list once to count how many times
    # each phrase occurs
    phrase_counts = defaultdict(int)
    for p in phrase_list:
        phrase_counts[p] += 1
    # and then a second time to compute RAKE scores
    for phrase in phrase_list:
        if phrase_counts[phrase] >= minFrequency:
            keyword_candidates.setdefault(phrase, 0)
            word_list = separate_words(phrase)
            candidate_score = 0
            for word in word_list:
                candidate_score += word_score[word]
            keyword_candidates[phrase] = candidate_score
    return keyword_candidates

def sort_tuple(tup):
    tup.sort(key = lambda x:x[1])
    return tup

class Rake(object):

    def __init__(self, stop_words, regex=r'[\W\n]+'):
        #lets users call predefined stopwords easily in a platform agnostic manner or use their own list
        if isinstance(stop_words, list):
            self.__stop_words_pattern = build_stop_word_regex(stop_words)
        else:
            self.__stop_words_pattern = build_stop_word_regex(load_stop_words(stop_words, regex))

    def run(self, text, minCharacters=2, maxWords=4, minFrequency=2):
        sentence_list = split_sentences(text)

        phrase_list = generate_candidate_keywords(sentence_list, self.__stop_words_pattern, minCharacters, maxWords)

        word_scores = calculate_word_scores(phrase_list)

        keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_scores, minFrequency)

        sorted_keywords = sorted(keyword_candidates.items(), key=operator.itemgetter(1), reverse=True)
        return sorted_keywords

In [37]:
rake_object = Rake(merged_sw)

keywords = sort_tuple(rake_object.run(text))
print(keywords[(len(keywords)-10):])

[('pre-war america', 7.228438228438229), ('time travel romances', 7.233092712828165), ('time travel element', 7.483092712828165), ('rare planetary alignment', 7.607142857142857), ('lovely young lady', 8.03951367781155), ('northwest passage series', 8.138326585695006), ('like time travel books', 8.438473046779928), ('better time travel books', 8.505819985555437), ('enjoy time travel books', 8.851274531009983), ('21-year-old trailblazing grandmother', 9.625314465408806)]
